17.36 Portfolio optimization with buy/hold/sell recommendations

In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

In [2]:
m = 30
n = 20
nb = 6
nh = 7
ns = 7
L = 2
sigma = .1
np.random.seed(1)
A = 0.06*np.random.normal(0, 1, size=(m, n))
Sigma = A.T@A

In [12]:
w = cp.Variable(n)
objective = cp.Maximize(cp.sum(w[:nb]) - cp.norm(w[nb:-ns], 1) - cp.sum(w[-ns:]))
constraints = [
    cp.sum(w) == 1, 
    cp.norm(w, 1) <= L, 
    cp.quad_form(w, Sigma) <= sigma ** 2,
    w[:nb] >= 0,
    w[-ns:] <= 0
]
prob = cp.Problem(objective, constraints)
result = prob.solve()
w_opt = w.value
print('optimal:', w_opt)
print('R^{wc}(w^*):', objective.value)
print('\mu^{wc}:', [1] * nb + list(np.where(w_opt[nb:-ns]>0, -1, 1)) + [-1] * ns) # Convert True to -1 False to 1 using numpy.where

optimal: [ 2.46546726e-01  2.46848541e-01  1.30793121e-01  2.31212112e-01
  1.63920697e-01  1.11579129e-01  8.93657387e-10  2.54955372e-02
  1.15036327e-01 -2.09889640e-02  7.39707892e-10  9.89497660e-10
 -3.89079793e-10 -1.69778536e-01 -3.09376139e-09 -5.44390019e-10
 -2.42764877e-02 -1.46287557e-10 -5.47165085e-10 -5.63882015e-02]
R^{wc}(w^*): 1.2198227254209728
\mu^{wc}: [1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1]


In [7]:
# naive method
w = cp.Variable(n)
objective = cp.Maximize(w.T @ np.array([1] * nb + [0] * nh + [-1] * ns))
constraints = [
    cp.sum(w) == 1, 
    cp.norm(w, 1) <= L, 
    cp.quad_form(w, Sigma) <= sigma ** 2,
]
prob = cp.Problem(objective, constraints)
result = prob.solve()
w_naive = w.value
print(w_naive)
print(objective.value) # R^{wc}(w^{naive})

[ 2.57988795e-01  2.84433494e-01  1.26956711e-01  2.73097995e-01
  1.57354653e-01  1.32537559e-01  3.92054471e-02  3.35990910e-02
  1.01733628e-01 -9.02466121e-02  2.42503918e-02  3.74675624e-10
 -1.88545734e-02 -2.13452945e-01 -7.07406026e-10  3.12196010e-10
 -5.55812961e-02  6.88422306e-02  5.57701735e-10 -1.21864569e-01]
1.5544257857756039
